In [54]:
import pandas as pd
from fuzzywuzzy import fuzz

In [55]:
def extract_answer_from_choices(choices, index):
    choice_list = eval(choices)
    index = int(float(index))
    return choice_list[index]

In [56]:
def compare_answers(correct_index, correct_answer, ai_answer):
    # Preprocess answers
    correct_answer = correct_answer.lower().strip()
    ai_answer = str(ai_answer).lower().strip().replace('*','').replace('  ',' ')
    
    # Calculate fuzzy string similarity
    similarity_ratio = fuzz.ratio(correct_answer, ai_answer)
    
    # Check if the AI's answer contains the correct answer
    contains_correct = correct_answer in ai_answer
    
    # Check if the correct answer contains the AI's answer (for short AI answers)
    contains_ai = ai_answer in correct_answer
    
    # Combine checks (you can adjust these conditions)
    return similarity_ratio > 80 or contains_correct or contains_ai or str(correct_index) in ai_answer.split()

In [57]:
MODEL = "gemma_9b"

In [58]:
# Load your data
df = pd.read_excel(f'{MODEL}/result_inference_mmlu.xlsx')  # Replace with your actual file path

In [59]:
df['Correct Answer'] = df.apply(lambda row: extract_answer_from_choices(row['Choices'], row['Answer Dataset']), axis=1)
df['Good Answer'] = df.apply(lambda row: compare_answers(row['Answer Dataset'], row['Correct Answer'], row['Answer Json']), axis=1)

In [60]:
df.to_excel(f'{MODEL}/updated_data.xlsx', index=False)

print("Comparison completed. Results saved to 'updated_data.csv'.")

Comparison completed. Results saved to 'updated_data.csv'.


In [61]:
result = df.groupby('Subject')['Good Answer'].mean().reset_index()
result.columns = ['Subject', 'Average Good Answer']

print(result)

                                Subject  Average Good Answer
0                      abstract_algebra             0.272727
1                               anatomy             0.571429
2                             astronomy             0.375000
3                       business_ethics             0.272727
4                    clinical_knowledge             0.344828
5                       college_biology             0.500000
6                     college_chemistry             0.625000
7              college_computer_science             0.272727
8                   college_mathematics             0.181818
9                      college_medicine             0.409091
10                      college_physics             0.272727
11                    computer_security             0.363636
12                   conceptual_physics             0.423077
13                         econometrics             0.333333
14               electrical_engineering             0.375000
15               element

In [62]:
result.to_excel('test.xlsx')